In [23]:
# Import
import pandas as pd
import numpy as np
import hvplot.pandas
import lib
from pathlib import Path

from pandas.tseries.offsets import DateOffset

# Setting these options will allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)

## Import the finta Python library

In [24]:
# Import the finta Python library and the TA module
from finta import TA

## Import the CSV file and create the Pandas DataFrame

In [25]:
# Read in CSV file in from the resources folder into a Pandas DataFrame
# Set the date as the DateTimeIndex
df = pd.read_csv(
    Path("../data/SP500_Data.csv"), 
    index_col="Date",
    infer_datetime_format=True, 
    parse_dates=True
    
)

# Review the DataFrame
df.head()

,Price,Open,High,Low,Change %
Date,,,,,
2002-07-03,954.0,948.1,954.3,934.9,0.62%
2002-07-05,989.0,965.8,989.1,954.0,3.67%
2002-07-08,977.0,989.0,993.6,972.9,-1.21%
2002-07-09,952.8,977.0,979.6,951.7,-2.48%
2002-07-10,920.5,952.8,956.3,920.3,-3.39%


In [26]:
# Plot the DataFrame with hvplot
# Calculate the daily returns using the closing prices and the pct_change function
df["actual_returns"] = df["Price"].pct_change()
df["actual_returns"].hvplot()

:Curve   [Date]   (actual_returns)

### Create a new trading algorithm using the Bollinger Bands technical indicator from the finta library. 

In [27]:
# Create a new clean copy of the last 5 years data

short_testing_begin = df.index.max() - DateOffset(years=5)
bb_signals_df = df.loc[short_testing_begin:]
 

In [28]:
bb_signals_df.columns=["close","open","high","low","change %", "actual_returns"]
# Review the DataFrame
bb_signals_df.head()

,close,open,high,low,change %,actual_returns
Date,,,,,,
2017-05-11,2394.44,2394.84,2395.72,2381.74,-0.22%,-0.002163
2017-05-12,2390.90,2392.44,2392.44,2387.19,-0.15%,-0.001478
2017-05-15,2402.32,2393.98,2404.05,2393.94,0.48%,0.004776
2017-05-16,2400.67,2404.55,2405.77,2396.05,-0.07%,-0.000687
2017-05-17,2357.03,2382.95,2384.87,2356.21,-1.82%,-0.018178


In [29]:
# Determine the Bollinger Bands for the Dataset
bbands_df = TA.BBANDS(bb_signals_df)

# Review the DataFrame
bbands_df.head()

,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,
2017-05-11,NaN,NaN,NaN
2017-05-12,NaN,NaN,NaN
2017-05-15,NaN,NaN,NaN
2017-05-16,NaN,NaN,NaN
2017-05-17,NaN,NaN,NaN


In [30]:
# Concatenate the Bollinger Bands to the DataFrame
bb_signals_df = pd.concat([bb_signals_df, bbands_df], axis=1)

# Review the DataFrame
bb_signals_df.iloc[50:60, :]

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER
Date,,,,,,,,,
2017-07-24,2469.91,2472.04,2473.10,2466.32,-0.11%,-0.001064,2483.898321,2442.5760,2401.253679
2017-07-25,2477.13,2477.88,2481.24,2474.91,0.29%,0.002923,2488.536489,2444.4790,2400.421511
2017-07-26,2477.83,2479.97,2481.69,2474.94,0.03%,0.000283,2492.197072,2447.4015,2402.605928
2017-07-27,2475.42,2482.76,2484.04,2459.93,-0.10%,-0.000973,2495.503230,2449.1380,2402.772770
2017-07-28,2472.10,2469.12,2473.53,2464.66,-0.13%,-0.001341,2497.028203,2451.7580,2406.487797
2017-07-31,2470.30,2475.94,2477.96,2468.53,-0.07%,-0.000728,2498.027967,2454.1025,2410.177033
2017-08-01,2476.35,2477.10,2478.51,2471.14,0.24%,0.002449,2499.799654,2456.4695,2413.139346
2017-08-02,2477.57,2480.38,2480.38,2466.48,0.05%,0.000493,2501.491774,2458.7210,2415.950226
2017-08-03,2472.16,2476.03,2476.03,2468.85,-0.22%,-0.002184,2498.193748,2461.8415,2425.489252


## Create a trading algorithm using Bollinger Bands

In [31]:
# Set the Signal column
bb_signals_df["Signal"] = 0.0

# Create a value to hold the initial trade signal
trade_signal = 0

# Update the DataFrame Signal column 1 (entry) or -1 (exit) for a long position trading algorithm
# where 1 is when the Close price is less than the BB_LOWER window
# where -1 is when the Close price is greater the the BB_UPPER window
# Incorporate a conditional in the if-statement, to evaluate the value of the trade_signal so the algorithm 
# plots only 1 entry and exit point per cycle.
for index, row in bb_signals_df.iterrows():
    if (row["close"] < row["BB_LOWER"]) and (trade_signal < 1):
        bb_signals_df.loc[index, "Signal"] = 1.0
        trade_signal += 1
        
    if (row["close"] > row["BB_UPPER"]) and (trade_signal > 0):
        bb_signals_df.loc[index, "Signal"] = -1.0
        trade_signal = 0


# Review the DataFrame
bb_signals_df

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal
Date,,,,,,,,,,
2017-05-11,2394.44,2394.84,2395.72,2381.74,-0.22%,-0.002163,NaN,NaN,NaN,0.0
2017-05-12,2390.90,2392.44,2392.44,2387.19,-0.15%,-0.001478,NaN,NaN,NaN,0.0
2017-05-15,2402.32,2393.98,2404.05,2393.94,0.48%,0.004776,NaN,NaN,NaN,0.0
2017-05-16,2400.67,2404.55,2405.77,2396.05,-0.07%,-0.000687,NaN,NaN,NaN,0.0
2017-05-17,2357.03,2382.95,2384.87,2356.21,-1.82%,-0.018178,NaN,NaN,NaN,0.0
2017-05-18,2365.72,2354.69,2375.74,2352.72,0.37%,0.003687,NaN,NaN,NaN,0.0
2017-05-19,2381.73,2371.37,2389.06,2370.43,0.68%,0.006767,NaN,NaN,NaN,0.0
2017-05-22,2394.02,2387.21,2395.46,2386.92,0.52%,0.005160,NaN,NaN,NaN,0.0
2017-05-23,2398.42,2397.04,2400.85,2393.88,0.18%,0.001838,NaN,NaN,NaN,0.0


In [32]:
# Visualize entry position relative to close price
entry = bb_signals_df[bb_signals_df["Signal"] == 1.0]["close"].hvplot.scatter(
    color='green',
    marker='^',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize exit position relative to close price
exit = bb_signals_df[bb_signals_df["Signal"] == -1.0]["close"].hvplot.scatter(
    color='red',
    marker='v',
    size=200,
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = bb_signals_df[["close"]].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_upper = bb_signals_df[["BB_UPPER"]].hvplot(
    line_color='purple',
    ylabel='Price in $',
    width=1000,
    height=400
)


bb_middle = bb_signals_df[["BB_MIDDLE"]].hvplot(
    line_color='orange',
    ylabel='Price in $',
    width=1000,
    height=400
)

bb_lower = bb_signals_df[["BB_LOWER"]].hvplot(
    line_color='blue',
    ylabel='Price in $',
    width=1000,
    height=400
)


# Overlay plots
bbands_plot = security_close * bb_upper * bb_middle * bb_lower * entry * exit
bbands_plot

:Overlay
   .Curve.I          :Curve   [Date]   (close)
   .Curve.II         :Curve   [Date]   (BB_UPPER)
   .Curve.III        :Curve   [Date]   (BB_MIDDLE)
   .Curve.IV         :Curve   [Date]   (BB_LOWER)
   .Scatter.Close.I  :Scatter   [Date]   (close)
   .Scatter.Close.II :Scatter   [Date]   (close)

## Backtest the Trading Strategy with 10 share and 100K initial capital

In [33]:
signals_df = bb_signals_df.copy()
# Set the initial capital
initial_capital = float(100000)

# Set the share size
share_size = 10

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df["Position"] = share_size * signals_df["Signal"]

# Find the points in time where a 100 share position is bought or sold
signals_df["Entry/Exit Position"] = signals_df["Position"]

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df["Portfolio Holdings"] = (
    signals_df["close"] * signals_df["Entry/Exit Position"].cumsum()
)

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df["Portfolio Cash"] = (
    initial_capital - (signals_df["close"] * signals_df["Entry/Exit Position"]).cumsum()
)

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df["Portfolio Total"] = (
    signals_df["Portfolio Cash"] + signals_df["Portfolio Holdings"]
)

# Calculate the portfolio daily returns
signals_df["strategy_returns"] = signals_df["Portfolio Total"].pct_change()

# Calculate the cumulative returns
signals_df["cumulative_Returns"] = (
    1 + signals_df["strategy_returns"]
).cumprod() - 1

# Print the DataFrame
signals_df.head()
#signals_df.to_csv("returns.csv")

In [34]:
# Visualize exit positions relative to total portfolio value
entry = signals_df[signals_df["Signal"] == 1.0]["Portfolio Total"].hvplot.scatter(
    color='purple',
    marker='^',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize entry positions relative to total portfolio value
exit = signals_df[signals_df["Signal"] == -1.0]["Portfolio Total"].hvplot.scatter(
    color='red',
    marker='v',
    legend=False, 
    ylabel="Total Portfolio Value", 
    width=1000, 
    height=400
)

# Visualize the total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay the plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(
    title="Bollinger Algorithm - Total Portfolio Value",
    yformatter='%.0f'
)

:Overlay
   .Curve.I                    :Curve   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [Date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [Date]   (Portfolio Total)

In [35]:
signals_df["Portfolio Total"].hvplot()

:Curve   [Date]   (Portfolio Total)

In [36]:
portfolio_evaluation_df = lib.performance_metrics(signals_df, "Bollinger_100K")

portfolio_evaluation_df.head()

,Bollinger_100K
Annualized Return,0.030466
Cumulative Returns,0.157124
Annual Volatility,1.068753
Sharpe Ratio,0.608686
Sortino Ratio,0.841106


In [37]:
portfolio_evaluation_df.to_csv("../data/Strategy_Bollinger.csv")

In [38]:
# Calculate weekly returns
weekly_returns = signals_df["cumulative_Returns"].groupby(
        by=[
            signals_df.index.year,
            signals_df.index.week
        ]
    ).cumsum()

# Display results    
weekly_returns.head()

C:\Users\padma\anaconda3\envs\algotrading\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: weekofyear and week have been deprecated, please use DatetimeIndex.isocalendar().week instead, which returns a Series.  To exactly reproduce the behavior of week and weekofyear and return an Index, you may call pd.Int64Index(idx.isocalendar().week)
  """


Date
2017-05-11    NaN
2017-05-12    0.0
2017-05-15    0.0
2017-05-16    0.0
2017-05-17    0.0
Name: cumulative_Returns, dtype: float64

In [39]:
weekly_returns.hvplot()

:Curve   [Date]   (cumulative_Returns)

In [40]:
weekly_returns.describe()

count    1259.000000
mean        0.231270
std         0.244099
min        -0.006550
25%         0.048920
50%         0.151794
75%         0.335704
max         1.106614
Name: cumulative_Returns, dtype: float64

In [41]:
bb_signals_df.head()

,close,open,high,low,change %,actual_returns,BB_UPPER,BB_MIDDLE,BB_LOWER,Signal
Date,,,,,,,,,,
2017-05-11,2394.44,2394.84,2395.72,2381.74,-0.22%,-0.002163,NaN,NaN,NaN,0.0
2017-05-12,2390.90,2392.44,2392.44,2387.19,-0.15%,-0.001478,NaN,NaN,NaN,0.0
2017-05-15,2402.32,2393.98,2404.05,2393.94,0.48%,0.004776,NaN,NaN,NaN,0.0
2017-05-16,2400.67,2404.55,2405.77,2396.05,-0.07%,-0.000687,NaN,NaN,NaN,0.0
2017-05-17,2357.03,2382.95,2384.87,2356.21,-1.82%,-0.018178,NaN,NaN,NaN,0.0


In [42]:
predictions_df = pd.DataFrame(index=bb_signals_df.index)
predictions_df["predicted_signal"] = bb_signals_df["Signal"]
predictions_df["actual_returns"] = bb_signals_df["actual_returns"]
# Calculate the  strategy returns
predictions_df["strategy_returns"] = predictions_df["actual_returns"] * predictions_df["predicted_signal"]
# Calculate the cumulative returns
predictions_df["cumulative_Returns"] = (
    1 + predictions_df["strategy_returns"]
).cumprod() -1

predictions_df.head()

,predicted_signal,actual_returns,strategy_returns,cumulative_Returns
Date,,,,
2017-05-11,0.0,-0.002163,-0.0,0.0
2017-05-12,0.0,-0.001478,-0.0,0.0
2017-05-15,0.0,0.004776,0.0,0.0
2017-05-16,0.0,-0.000687,-0.0,0.0
2017-05-17,0.0,-0.018178,-0.0,0.0


In [43]:
portfolio_evaluation_df = lib.performance_metrics(predictions_df, "Bollinger")

portfolio_evaluation_df.head()

,Bollinger
Annualized Return,-0.084197
Cumulative Returns,-0.346558
Annual Volatility,1.629373
Sharpe Ratio,-2.01421
Sortino Ratio,-1.99897


In [44]:
(1 + predictions_df[["actual_returns", "strategy_returns"]]).cumprod().hvplot()

:NdOverlay   [Variable]
   :Curve   [Date]   (value)